In [64]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

import sklearn.metrics as metrics

from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

In [65]:
df = pd.read_csv('data/concat_dfs.csv')
adj_df = pd.read_csv('data/adjusted_class.csv')

In [66]:
df.head()

,Rank,Account,Character,Class,Level,Experience,Dead,Atlas_Invasion,Delirium_Everywhere,Endless_Delve,Endless_Heist,Expedition,Standard,Scourge,Ultimatum,Character Count
0,1,Kiom,qazwsxedcrfvtgbyhnujmi,Saboteur,100,4250334444,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6
1,2,eliehh,Eliot_invaded,Saboteur,100,4250334444,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,19
2,3,김겨울,DD는원래사기였다ㆍ김겨울ㆍ따라잡지말아주세요,Necromancer,99,3959950552,1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20
3,4,Akiobird,RhiㆍUnlucky,Necromancer,99,3953455777,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6
4,5,Senizen,Senvasion,Inquisitor,99,3936854161,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12


In [67]:
#What do I really want out of this?

#Supervised
#Predict global account survival rates --> needs test data
#Predict character class survival rates --> needs test data, consider ascended vs base?
#Predict character performance --> needs test data

#Unsupervised
#Group account performance (are there accounts who consistently achieve high performance (level) across leagues? Low performance?)
#Group account survival (are there accounts with multiple characters w/o deaths?)

In [68]:
adj_df =pd.get_dummies(adj_df, columns=['Class', 'Account'])
adj_df.head()

,Rank,Character,Level,Experience,Dead,Atlas_Invasion,Delirium_Everywhere,Endless_Delve,Endless_Heist,Expedition,...,Account_코코밍밍,Account_퀄리어틴,Account_타다라,Account_테라스z,Account_퇴근희망,Account_티모유미,Account_판타라이,Account_하이델른,Account_형준39,Account_화랑칸
0,1,qazwsxedcrfvtgbyhnujmi,100,4250334444,0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
1,2,Eliot_invaded,100,4250334444,0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
2,3,DD는원래사기였다ㆍ김겨울ㆍ따라잡지말아주세요,99,3959950552,1,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
3,4,RhiㆍUnlucky,99,3953455777,0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0
4,5,Senvasion,99,3936854161,0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0


In [69]:
col_names = []
for col in adj_df.columns:
    if 'Account' in col or 'Class' in col:
        col_names.append(col)

features = ['Level', 'Dead'] + col_names
X = adj_df[features]

ss = StandardScaler()
X_scaled = ss.fit_transform(X)

km = KMeans(n_clusters=8, random_state=42)
km.fit(X_scaled)

KMeans(random_state=42)

In [70]:
adj_df['cluster'] = km.labels_
adj_df.head()

,Rank,Character,Level,Experience,Dead,Atlas_Invasion,Delirium_Everywhere,Endless_Delve,Endless_Heist,Expedition,...,Account_퀄리어틴,Account_타다라,Account_테라스z,Account_퇴근희망,Account_티모유미,Account_판타라이,Account_하이델른,Account_형준39,Account_화랑칸,cluster
0,1,qazwsxedcrfvtgbyhnujmi,100,4250334444,0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,5
1,2,Eliot_invaded,100,4250334444,0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,5
2,3,DD는원래사기였다ㆍ김겨울ㆍ따라잡지말아주세요,99,3959950552,1,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,5
3,4,RhiㆍUnlucky,99,3953455777,0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,5
4,5,Senvasion,99,3936854161,0,1.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,5


In [71]:
adj_df.drop(columns=col_names).groupby('cluster').mean()

,Rank,Level,Experience,Dead,Atlas_Invasion,Delirium_Everywhere,Endless_Delve,Endless_Heist,Expedition,Standard,Scourge,Ultimatum,Character Count
cluster,,,,,,,,,,,,,
0,1656.333333,78.333333,1.657013e+09,0.333333,0.000000,0.333333,0.000000,0.000000,0.666667,0.000000,0.000000,0.000000,3.000000
1,1371.500000,92.250000,2.426206e+09,0.500000,0.000000,0.000000,0.500000,0.000000,0.250000,0.000000,0.250000,0.000000,4.000000
2,664.285714,80.857143,1.057465e+09,1.000000,0.285714,0.571429,0.000000,0.142857,0.000000,0.000000,0.000000,0.000000,7.000000
3,1290.000000,90.500000,2.084469e+09,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
4,1925.000000,43.000000,2.823394e+07,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
5,1000.474047,87.536598,2.240954e+09,0.630956,0.125039,0.124851,0.124914,0.125164,0.124977,0.125227,0.124977,0.124851,5.050466
6,935.666667,95.333333,3.030344e+09,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.666667,3.000000
7,779.111111,94.000000,2.869747e+09,0.888889,0.111111,0.000000,0.111111,0.000000,0.111111,0.000000,0.222222,0.444444,9.000000


In [72]:
centroids = ss.inverse_transform(km.cluster_centers_)
centroids

array([[7.83333333e+01, 3.33333333e-01, 3.33333333e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [9.22500000e+01, 5.00000000e-01, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [8.08571429e+01, 1.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 1.35525272e-20],
       ...,
       [8.75365976e+01, 6.30956108e-01, 2.59407676e-01, ...,
        1.25226974e-04, 6.26134869e-05, 6.26134869e-05],
       [9.53333333e+01, 1.00000000e+00, 1.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [9.40000000e+01, 8.88888889e-01, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [73]:
km.inertia_

110783781.07686342

In [74]:
silhouette_score(X_scaled, km.labels_)

-0.2267829737629656

In [75]:
df =pd.get_dummies(df, columns=['Class', 'Account'])

In [76]:
col_names2 = []
for col in df.columns:
    if 'Account' in col or 'Class' in col:
        col_names2.append(col)

features2 = ['Level', 'Dead'] + col_names2
X2 = df[features2]

ss2 = StandardScaler()
X_scaled2 = ss2.fit_transform(X2)

km2 = KMeans(n_clusters=8, random_state=42)
km2.fit(X_scaled2)

KMeans(random_state=42)

In [78]:
df['cluster'] = km2.labels_

In [79]:
df.drop(columns=col_names2).groupby('cluster').mean()

,Rank,Level,Experience,Dead,Atlas_Invasion,Delirium_Everywhere,Endless_Delve,Endless_Heist,Expedition,Standard,Scourge,Ultimatum,Character Count
cluster,,,,,,,,,,,,,
0,1271.000000,72.000000,4.540284e+08,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
1,963.882038,84.772118,1.974648e+09,0.613941,0.158177,0.182306,0.142091,0.187668,0.037534,0.144772,0.040214,0.107239,5.243968
2,1021.214308,86.287658,2.040566e+09,0.953970,0.132773,0.139045,0.134793,0.129159,0.157436,0.023599,0.135644,0.147550,4.694483
3,1376.666667,96.333333,3.360904e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.333333,0.666667,0.000000,0.000000,3.000000
4,841.200000,96.800000,3.450910e+09,0.600000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.800000,5.000000
5,241.750000,92.000000,2.345526e+09,1.000000,0.500000,0.250000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,4.000000
6,971.570277,89.578820,2.558578e+09,0.142811,0.111219,0.100419,0.109284,0.115087,0.081238,0.277402,0.114281,0.091070,5.585590
7,1237.666667,95.000000,2.909188e+09,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,3.000000


In [80]:
km2.inertia_

111072079.67130217

In [81]:
silhouette_score(X_scaled2, km2.labels_)

-0.1463232281566964

In [ ]:
inertia_list = []

for k in range(1, 20):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_scaled)
    inertia_list.append(kmeans.inertia_)
    
inertia_list

In [ ]:
plt.plot(range(1, 20), inertia_list, marker='o')
plt.xlabel('# of Clusters')
plt.ylabel('Score')
plt.title('Inertia Scores');

In [ ]:
silhouette_list = []

for k in range(2, 20):
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_scaled)
    silhouette_list.append(silhouette_score(X_scaled, kmeans.labels_))
silhouette_list

In [ ]:
plt.plot(range(2, 20), silhouette_list, marker='o')
plt.xlabel('# of Clusters')
plt.ylabel('Score')
plt.title('Silhouette Scores');

In [ ]:
dbscan = DBSCAN()
dbscan.fit(X_scaled);

In [ ]:
silhouette_score(X_scaled, dbscan.labels_)

# Elbow Curve